In [5]:
import covalent as ct


In [6]:
!pip install ase
!pip install ase-notebook
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate
from ase.io import read
from ase.io.trajectory import Trajectory
from ase_notebook import AseView, ViewConfig
from matplotlib import pyplot as plt


/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


ImportError: cannot import name 'AseView' from 'ase_notebook' (/home/aravindprabaharan/.local/lib/python3.8/site-packages/ase_notebook/__init__.py)

In [ ]:
@ct.electron
def construct_cu_slab(
    unit_cell=(4, 4, 2),
    vacuum=10.0,
):
    slab = fcc111("Cu", size=unit_cell, vacuum=vacuum)
    return slab


@ct.electron
def compute_system_energy(system):
    system.calc = EMT()
    return system.get_potential_energy()


@ct.electron
def construct_n_molecule(d=0):
    return Atoms("2N", positions=[(0.0, 0.0, 0.0), (0.0, 0.0, d)])


@ct.electron
def get_relaxed_slab(slab, molecule, height=1.85):
    slab.calc = EMT()
    add_adsorbate(slab, molecule, height, "ontop")
    constraint = FixAtoms(mask=[a.symbol != "N" for a in slab])
    slab.set_constraint(constraint)
    dyn = QuasiNewton(slab, trajectory="/tmp/N2Cu.traj", logfile="/tmp/temp")
    dyn.run(fmax=0.01)
    return slab


In [ ]:
N2 = construct_n_molecule(d=1.2)
slab = construct_cu_slab(unit_cell=(4, 4, 2), vacuum=10.0)
add_adsorbate(slab, N2, 1.2, "ontop")

config = ViewConfig(show_unit_cell=False)
ase_view = AseView(config)
ase_view.config.rotations = "-90x,50y,0z"
config.atom_lighten_by_depth = 0.7
ase_view.make_svg(
    slab,
    center_in_uc=True,
    repeat_uc=(2, 2, 1),
)


In [ ]:
@ct.lattice
def compute_energy(initial_height=3, distance=1.10):
    N2 = construct_n_molecule(d=distance)
    e_N2 = compute_system_energy(system=N2)

    slab = construct_cu_slab(unit_cell=(4, 4, 2), vacuum=10.0)
    e_slab = compute_system_energy(system=slab)

    relaxed_slab = get_relaxed_slab(slab=slab, molecule=N2, height=initial_height)
    e_relaxed_slab = compute_system_energy(system=relaxed_slab)
    final_result = e_slab + e_N2 - e_relaxed_slab

    return final_result


In [ ]:
compute_energy.draw(initial_height=3, distance=1.10)


In [ ]:
from IPython import display
display.Image("graphs/compute_energy.png")

In [ ]:
dispatch_id = ct.dispatch(compute_energy)(initial_height=3, distance=1.10)


In [ ]:
result = ct.get_result(dispatch_id=dispatch_id, wait=True)

print(
    f"Computation status = {result.status}\nEnergy of slab + nitrogen system = {result.result:.4f}"
)


In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=1)
db = ct.get_data_store()

pp.pprint(result.get_all_node_outputs())


In [ ]:
pp.pprint(result.get_node_result(node_id=0))


In [ ]:
import numpy as np

optimize_height = ct.electron(compute_energy)


@ct.lattice
def vary_distance(distance):
    result = []
    for i in distance:
        result.append(optimize_height(initial_height=3, distance=i))
    return result


In [ ]:
import numpy as np

distance = np.linspace(1, 1.5, 3)

vary_distance.draw(distance=np.round(distance, 2))


In [ ]:
display.Image("graphs/vary_distance.png")

In [ ]:
seps = 7
distance = np.linspace(1, 1.5, seps)
dispatch_id = ct.dispatch(vary_distance)(distance=distance)


In [ ]:
result = ct.get_result(dispatch_id, wait=True)
print(result.status)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), facecolor="w")
ax.plot(distance, result.result, marker="o")
ax.set_xlabel("Nitrogen molecule spacing")
ax.set_ylabel("Structural relaxation energy of Cu + ${N}_2$")
plt.tight_layout()
plt.show()
